In [27]:
import pathlib

import duckdb
import pandas as pd

In [28]:
result_path = pathlib.Path("../../2.cellprofiling/results/C4-2/").resolve(strict=True)
# get a list of all parquets in the directory
parquet_files = list(result_path.glob("*.parquet"))
parquet_files.sort()

In [51]:
pd.options.display.max_columns = None

In [82]:
feature_types_dict = {
    "Organoid": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Cell": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Nuclei": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
    "Cytoplasm": {
        "AreaSize_Shape": [],
        "Colocalization": [],
        "Intensity": [],
        "Granularity": [],
        "Neighbor": [],
        "Texture": [],
    },
}
for file in parquet_files:
    for compartment in feature_types_dict.keys():
        for feature_type in feature_types_dict[compartment].keys():
            if compartment in file.name and feature_type in file.name:
                feature_types_dict[compartment][feature_type].append(file)
feature_types_dict.keys()

dict_keys(['Organoid', 'Cell', 'Nuclei', 'Cytoplasm'])

In [83]:
# create a record for each compartment
merged_df_dict = {
    "Organoid": [],
    "Cell": [],
    "Nuclei": [],
    "Cytoplasm": [],
}

In [119]:
df = pd.read_parquet(feature_types_dict["Nuclei"]["Intensity"][-1])
# melt to tidy wide
df["feature"] = (
    "Intensity_"
    + df["Nuclei_Mito_compartment"]
    + "_"
    + df["Nuclei_Mito_channel"]
    + "_"
    + df["Nuclei_Mito_feature_name"]
)
df.drop(
    columns=[
        "Nuclei_Mito_compartment",
        "Nuclei_Mito_channel",
        "Nuclei_Mito_feature_name",
    ],
    inplace=True,
)
df.head(1)

,Nuclei_Mito_object_id,Nuclei_Mito_value,image_set,feature
0,1,123182414,C4-2,Intensity_Nuclei_Mito_INTEGRATED.INTENSITY


In [120]:
# pivot to wide
df = df.pivot(
    index=["Nuclei_Mito_object_id", "image_set"],
    columns="feature",
    values="Nuclei_Mito_value",
)
df.reset_index(inplace=True)
# df.drop(columns=['feature'], inplace=True)
df.rename(columns={"Nuclei_Mito_object_id": "object_id"}, inplace=True)
df.head()

feature,object_id,image_set,Intensity_Nuclei_Mito_CM.X,Intensity_Nuclei_Mito_CM.Y,Intensity_Nuclei_Mito_CM.Z,Intensity_Nuclei_Mito_CMI.X,Intensity_Nuclei_Mito_CMI.Y,Intensity_Nuclei_Mito_CMI.Z,Intensity_Nuclei_Mito_DIFF.X,Intensity_Nuclei_Mito_DIFF.Y,Intensity_Nuclei_Mito_DIFF.Z,Intensity_Nuclei_Mito_EDGE.COUNT,Intensity_Nuclei_Mito_I.X,Intensity_Nuclei_Mito_I.Y,Intensity_Nuclei_Mito_I.Z,Intensity_Nuclei_Mito_INTEGRATED.INTENSITY,Intensity_Nuclei_Mito_INTEGRATED.INTENSITY.EDGE,Intensity_Nuclei_Mito_LOWER.QUARTILE.INTENSITY,Intensity_Nuclei_Mito_MAD.INTENSITY,Intensity_Nuclei_Mito_MASS.DISPLACEMENT,Intensity_Nuclei_Mito_MAX.INTENSITY,Intensity_Nuclei_Mito_MAX.INTENSITY.EDGE,Intensity_Nuclei_Mito_MAX.X,Intensity_Nuclei_Mito_MAX.Y,Intensity_Nuclei_Mito_MAX.Z,Intensity_Nuclei_Mito_MEAN.INTENSITY,Intensity_Nuclei_Mito_MEAN.INTENSITY.EDGE,Intensity_Nuclei_Mito_MEDIAN.INTENSITY,Intensity_Nuclei_Mito_MIN.INTENSITY,Intensity_Nuclei_Mito_MIN.INTENSITY.EDGE,Intensity_Nuclei_Mito_STD.INTENSITY,Intensity_Nuclei_Mito_STD.INTENSITY.EDGE,Intensity_Nuclei_Mito_UPPER.QUARTILE.INTENSITY,Intensity_Nuclei_Mito_VOLUME
0,1,C4-2,769,768,16,487,486,10,281,281,5,1770,60105906630,59988741120,1249765440,123182414,27082448,6252,4605,397,65495,60823,1390,410,0,9985,15300,7577,3631,5151,8692,11844,9347,12336
1,2,C4-2,769,768,16,158,157,3,611,610,12,6498,60105906630,59988741120,1249765440,379688900,31048132,3902,2146,863,36494,14857,1390,410,0,2146,4778,4176,2403,3130,593,705,4615,176884
2,3,C4-2,769,768,16,153,153,3,615,614,12,7004,60105906630,59988741120,1249765440,390414025,33322664,4092,2978,869,52513,52513,1390,410,0,1489,4757,4380,2680,2883,684,899,4766,262164
3,4,C4-2,769,768,16,179,178,3,590,589,12,5760,60105906630,59988741120,1249765440,335576440,26871968,3845,3111,834,61423,64507,1390,410,0,1037,4665,4106,2374,2697,862,3766,4386,323540
4,5,C4-2,769,768,16,444,443,9,325,324,6,4046,60105906630,59988741120,1249765440,135326652,13852665,2998,2668,459,7735,7735,1390,410,0,667,3423,3268,1833,1835,439,481,3612,202865


In [69]:
df = pd.read_parquet(feature_types_dict["AreaSize_Shape"][0])
df_dict = {
    "organoid": [],
    "nuclei": [],
    "cell": [],
    "cytoplasm": [],
}
for areasizeshape_df in feature_types_dict["AreaSize_Shape"]:
    df = pd.read_parquet(areasizeshape_df)
    # rename any column that contain object id to "ObjectID"
    for col in df.columns:
        if "object_id" in col:
            df.rename(columns={col: "ObjectID"}, inplace=True)
        elif "image_set" in col:
            continue
        else:
            # prepend the column name with the feature type
            df.rename(columns={col: f"AreaSizeShape_{col}"}, inplace=True)
    if "Organoid" in areasizeshape_df.stem:
        df_dict["organoid"].append(df)
    elif "Nuclei" in areasizeshape_df.stem:
        df_dict["nuclei"].append(df)
    elif "Cell" in areasizeshape_df.stem:
        df_dict["cell"].append(df)
    elif "Cytoplasm" in areasizeshape_df.stem:
        df_dict["cytoplasm"].append(df)

# merge all of the organoid dataframes into one by ObjectID and image_set with duckdb
con = duckdb.connect()
dfs = {}
for i, df in enumerate(df_dict["organoid"]):
    dfs[f"{i}"] = df

In [75]:
df

,ObjectID,AreaSizeShape_Organoid_Mito_VOLUME,AreaSizeShape_Organoid_Mito_CENTER.X,AreaSizeShape_Organoid_Mito_CENTER.Y,AreaSizeShape_Organoid_Mito_CENTER.Z,AreaSizeShape_Organoid_Mito_BBOX.VOLUME,AreaSizeShape_Organoid_Mito_MIN.X,AreaSizeShape_Organoid_Mito_MAX.X,AreaSizeShape_Organoid_Mito_MIN.Y,AreaSizeShape_Organoid_Mito_MAX.Y,AreaSizeShape_Organoid_Mito_MIN.Z,AreaSizeShape_Organoid_Mito_MAX.Z,AreaSizeShape_Organoid_Mito_EXTENT,AreaSizeShape_Organoid_Mito_EULER.NUMBER,AreaSizeShape_Organoid_Mito_EQUIVALENT.DIAMETER,AreaSizeShape_Organoid_Mito_SURFACE.AREA,image_set
0,1,20948726.0,663.314682,552.593818,15.26746,33991848.0,124,1152,45,1047,0,33,0.616287,1,342.021178,NaN,C4-2


In [74]:
def duckdb_merge(dfs: dict, key: str = "id", join_type: str = "INNER") -> pd.DataFrame:
    """
    Dynamically merges multiple DataFrames using DuckDB on a common key.

    Parameters:
    - dfs (dict): Dictionary of {name: DataFrame}
    - key (str): The common key to join on
    - join_type (str): Join type: INNER, LEFT, RIGHT, FULL

    Returns:
    - Merged pandas DataFrame
    """
    con = duckdb.connect()

    # Register DataFrames in DuckDB
    for name, df in dfs.items():
        con.register(name, df)

    # Start building the SQL query
    table_names = list(dfs.keys())
    base = table_names[0]
    query = f"SELECT * FROM {base} "

    for tbl in table_names[1:]:
        query += f"{join_type} JOIN {tbl} USING({key}) "

    return con.execute(query).fetchdf()


duckdb_merge(
    dfs,
    key="ObjectID",
    # join_type="INNER",
)

ParserException: Parser Error: syntax error at or near "0"